In [ ]:
import asyncio 
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.utils.keyboard import ReplyKeyboardBuilder
from aiogram import F
from dotenv import load_dotenv
import os

logging.basicConfig(level=logging.INFO) 

load_dotenv(dotenv_path = "proj/token.env")

bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
#api_key = os.getenv("OPENAI_API_KEY")

bot = Bot(token=bot_token)
dp = Dispatcher()

#openai.api_key = api_key

conv_hist = {}

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    builder = ReplyKeyboardBuilder()
    builder.add(types.KeyboardButton(text="GPT"))
    builder.add(types.KeyboardButton(text="DeepSeek"))
    builder.add(types.KeyboardButton(text="Claude"))
    builder.add(types.KeyboardButton(text="Kandinsky"))
    builder.adjust(2)
    await message.answer("Какую модель Вы хотите использовать?", reply_markup=builder.as_markup(resize_keyboard=True, one_time_keyboard=True))

@dp.message((F.text.lower() == "deepseek") | (F.text.lower() == "claude") | (F.text.lower() == "kandinsky"))
async def deepseek(message: types.Message):
    await message.answer("К сожалению, этот вариант еще находится в разработке. Попробуйте использовать GPT.")
    
@dp.message(F.text.lower() == "gpt")
async def gptchat(message: types.Message):
    chat_id = message.chat.id
    if chat_id not in conv_hist:
        conv_hist[chat_id] = []
        
    conv_hist[chat_id].append({"role": "user", "content": message.text})
    
    response = openai.ChatCompletion.create(
        #model = ""
        messages = conv_hist[chat_id]
    )

    bot_reply = respomse["choices"][0]["message"]["content"]
    conv_hist[chat_id].append({"role": "assistant", "content": bot_reply})
    
    await message.reply(bot_reply)

async def main():
    await dp.start_polling(bot)

await main()

if __name__ == "__main__":
    asyncio.run(main())